In [ ]:
import glob
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder,  DatasetFolder,VisionDataset
from torchvision.transforms import Compose, Normalize, Resize, ToTensor 

from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

import os

from torchvision.models import vgg19

import helper
import torchvision.models as models

import pickle
%matplotlib inline
warnings.filterwarnings("ignore")
print(os.listdir("../input"))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
#!nvidia-smi

In [ ]:
print(torch.cuda.is_available())

In [ ]:
#conda install -c anaconda cudatoolkit
#!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

In [ ]:
# Different classes dataset. 
classes = ('1','3','0')

In [ ]:
data_path = '../input/neurowood/trainset/'
#test_path = '../input/woodhack/test1' # predictions on this dataset should be submitted for the competition 



## Image Augmentations

In [ ]:
# For vallidation and test data

transformations = transforms.Compose([
         #transforms.ToPILImage(),
         transforms.Resize((512, 512)),
         transforms.CenterCrop(350),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]) # это среднее и стандартное отклонение всего датасета (обычно imagenet), на котором обучали большую сеть
    ])



# For train data
transformations2 = transforms.Compose([
         transforms.Resize((512, 512)),
         transforms.CenterCrop(350),
        transforms.ColorJitter(brightness=0.2),
         transforms.RandomGrayscale(p=0.2),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=(-20,+20)),
         transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])
transformations3 = transforms.Compose([
         transforms.Resize((512, 512)),
         transforms.RandomCrop(350),
       transforms.RandomInvert(p=0.4),
         transforms.RandomGrayscale(p=0.2),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=(-40,+40)),
         transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])
transformations4 = transforms.Compose([
         transforms.Resize((512, 512)),
         transforms.RandomCrop(350),
        transforms.ColorJitter(brightness=0.5),
         transforms.RandomGrayscale(p=0.2),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(degrees=(-40,+40)),
         transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])    

transformations5 = transforms.Compose([
         transforms.Resize((512, 512)),
         transforms.CenterCrop(350),
        transforms.RandomRotation(degrees=15),
         transforms.RandomGrayscale(p=0.15),
        transforms.RandomHorizontalFlip(p=0.15),
        transforms.RandomVerticalFlip(p=0.15),
         transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ])      

In [ ]:
# data = {
#     'train':
#     datasets.ImageFolder(root=data_path, transform=image_transforms['train']),
#     'valid':
#     datasets.ImageFolder(root=validdir, transform=image_transforms['valid']),
#     'test':
#     datasets.ImageFolder(root=testdir, transform=image_transforms['test'])
# }
# # Dataloader iterators, make sure to shuffle
# dataloaders = {
#     'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True,num_workers=10),
#     'val': DataLoader(data['valid'], batch_size=batch_size, shuffle=True,num_workers=10),
#     'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True,num_workers=10)
# }

In [ ]:
# class WoodDataset(Dataset):
    

#     def __init__(self, root_dir, transform=None):
#         """
#         Args:
#             root_dir (string): Directory with all the images.
#             transform (callable, optional): Optional transform to be applied
#                 on a sample.
#         """
#         self.root_dir = root_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.landmarks_frame)

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()

#         img_name = os.path.join(self.root_dir,
#                                 self.landmarks_frame.iloc[idx, 0])
#         image = io.imread(img_name)
#         sample = {'image': image, 'landmarks': landmarks}

#         if self.transform:
#             sample = self.transform(sample)

#         return sample

In [ ]:
#Allset = ImageFolder(data_path,transform=transformations)
dataset1 = ImageFolder(data_path,transform=transformations)
dataset2 = ImageFolder(data_path, transform=transformations2)
dataset3 = ImageFolder(data_path, transform=transformations3)
dataset4 = ImageFolder(data_path, transform=transformations4)
dataset5 = ImageFolder(data_path, transform=transformations5)

dataset6 = dataset1 + dataset5

dataset = dataset1 + dataset2 + dataset3 + dataset4 + dataset5


#test_set = ImageFolder(test_path, transform=transformations)

In [ ]:
train, validation_set = torch.utils.data.random_split(
    dataset6, 
    [int(0.5 * len(dataset6)), len(dataset6) - int(0.5 * len(dataset6))]
)
print(f'training size: {len(train)}\ntest size: {len(validation_set)}')

In [ ]:
train_set = train + dataset2 + dataset3 + dataset4

In [ ]:
# train_set, validation_set = torch.utils.data.random_split(
#     dataset, 
#     [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))]
# )
# print(f'training size: {len(train_set)}\ntest size: {len(validation_set)}')

In [ ]:
#transformations(train_set)

In [ ]:
#  import requests
# from PIL import Image

# def im_convert(tensor):  
#   image = tensor.cpu().clone().detach().numpy() # This process will happen in normal cpu.
#   image = image.transpose(1, 2, 0)
#   image = image * np.array((0.485, 0.485, 0.485)) + np.array((0.229, 0.229, 0.229))
#   image = image.clip(0, 1)
#   return image

# # i = 100
# # path = "../input/woodhack/test1/test/" + str(i) + ".png"
# # img = Image.open(path)
# # #plt.imshow(img)
# # img = transformations(img)
# # #img = img[None, :]
# # #plt.imshow(im_convert(img))
# # #print(img.size())
# # #predicted = model(img)
# # plt.imshow(im_convert(img))

# #plt.imshow(im_convert(dataset1[1]))
# print(dataset1[1].size)

In [ ]:
batch_size = 64


train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=2)
validation_dataloader = DataLoader(validation_set, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=2)
#test_dataloader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True, num_workers=2)
len(train_dataloader.dataset)

## Efficientnet

In [ ]:
!pip install --upgrade efficientnet-pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet
model_name = 'efficientnet-b7'
model = EfficientNet.from_pretrained(model_name)  #, num_classes=3
model.eval()
#model



In [ ]:
# Чтобы заморозить параметры на всех слоях

for param in model.parameters():
    param.requires_grad = False 

In [ ]:
model._fc = torch.nn.Sequential(
    nn.Linear(in_features=2560, out_features=625, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=625, out_features=256, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=256, out_features=3, bias=True))
model.to(device)

In [ ]:
# PATH = "../input/woodhack-hackathon/effnetb7.pth"
# model.load_state_dict(torch.load(PATH))
# model.eval()


In [ ]:
pip install torch-summary

Model Summary

In [ ]:
from torchsummary import summary
model.to(device)
#summary(model, input_size = (3,224,224))

## googlenet 

In [ ]:
model_googlenet = models.googlenet(pretrained=True)
model_googlenet.eval()

In [ ]:
# Чтобы заморозить параметры на всех слоях

for param in model_googlenet.parameters():
    param.requires_grad = False 

In [ ]:
model_googlenet.fc = torch.nn.Sequential(
    nn.Linear(in_features=1024, out_features=256, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=256, out_features=3, bias=True))
model_googlenet.to(device)

In [ ]:
# PATH = "../input/woodhack-hackathon/googlenet.pth"
# model_googlenet.load_state_dict(torch.load(PATH))
model_googlenet.eval()

In [ ]:
summary(model_googlenet, input_size = (3,224,224))

## Pnasnet5large

In [ ]:
pip install timm

In [ ]:
import timm
model_pnasnet5large = timm.create_model('pnasnet5large', pretrained=True)
model_pnasnet5large.eval()

In [ ]:
# Чтобы заморозить параметры на всех слоях

for param in model_pnasnet5large.parameters():
    param.requires_grad = False 

In [ ]:
model_pnasnet5large.last_linear = torch.nn.Sequential(
    nn.Linear(in_features=4320, out_features=1024, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=1024, out_features=256, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=256, out_features=3, bias=True))
model_pnasnet5large.to(device)

In [ ]:
# PATH = "../input/woodhack-hackathon/pnas.pth"
# model_pnasnet5large.load_state_dict(torch.load(PATH))
# model_pnasnet5large.eval()

In [ ]:
#summary(model_pnasnet5large, input_size = (3,224,224))

## Inseption

In [ ]:
model_inception = models.resnext50_32x4d(pretrained=True)
model_inception.eval()

In [ ]:
for param in model_inception.parameters():
    param.requires_grad = False 
    

In [ ]:
model_inception.fc = torch.nn.Sequential(
    nn.Linear(in_features=2048, out_features=1024, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=1024, out_features=256, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=256, out_features=3, bias=True))
model_inception.to(device)

In [ ]:
# PATH = "../input/woodhack-hackathon/inseption.pth"
# model_inception.load_state_dict(torch.load(PATH))
# model_inception.eval()

# model_inception.to(device)

In [ ]:
summary(model_inception, input_size = (3,224,224))

## Densenet

In [ ]:
import timm
#timm.list_models()
model_densenet = timm.create_model('densenet264')
model_densenet.eval()

In [ ]:
for param in model_densenet.parameters():
    param.requires_grad = False 
    
model_densenet.classifier = torch.nn.Sequential(
    nn.Linear(in_features=4032, out_features=1024, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=1024, out_features=3, bias=True))

In [ ]:
# PATH = "../input/woodhack-hackathon/dencenet.pth"
# model_densenet.load_state_dict(torch.load(PATH))
# model_densenet.eval()

# model_densenet.to(device)

In [ ]:
summary(model_densenet, input_size = (3,224,224))

## Resnet152

In [ ]:
from torchvision.models import resnet152
import torchvision.models as models


# Загрузить предобученную сеть -- pretrained=True
model_resnet50 = models.resnet152(pretrained=True)
model_resnet50.eval()


for param in model_resnet50.parameters():
    param.requires_grad = False 
    
model_resnet50.fc = torch.nn.Sequential(
    nn.Linear(in_features=2048, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=256, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(in_features=256, out_features=3, bias=True))


## Загрузка с файла Resnet50

In [ ]:
# PATH = "../input/woodhack-hackathon/resnet50.pth"
# model_resnet50.load_state_dict(torch.load(PATH))
# model_resnet50.eval()

model_resnet50.to(device)

In [ ]:
summary(model_resnet50, input_size = (3,224,224))

In [ ]:
model_vgg19 = vgg19(pretrained=True)
model_vgg19.eval()


for param in model_vgg19.parameters():
    param.requires_grad = False 
    
model_vgg19.classifier = torch.nn.Sequential(
    nn.Linear(in_features=25088, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=256, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=256, out_features=3, bias=True))

In [ ]:
# PATH = "../input/woodhack-hackathon/vgg19.pth"
# model_vgg19.load_state_dict(torch.load(PATH))
# model_vgg19.eval()

model_vgg19.to(device)

In [ ]:
summary(model_vgg19, input_size = (3,224,224))

In [ ]:
def train_epoch(
    model,
    data_loader,
    optimizer,
    criterion,
    return_losses=False,
    device="cuda",
):
    model = model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    total_predictions = np.array([])#.reshape((0, ))
    total_labels = np.array([])#.reshape((0, ))
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            # Move Batch to GPU
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            # Update weights
            loss.backward()
            optimizer.step() #step - обновление весов модели
            optimizer.zero_grad() #zero_grad - занулить веса модели (по умолчанию градиенты в PyTorch аккумулируются) ~ for each param in params: param.grad = None
            # Update descirption for tqdm
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description( 
                f"Loss: {round(loss.item(), 4)} " # .item - from tensor to python number
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    if return_losses:
        return metrics, all_losses
    else:
        return metrics


def validate(model, data_loader, criterion, device="cuda"):
    model = model.eval() # выключает некоторые слои при подсчете
    total_loss = 0
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])
    with tqdm(total=len(data_loader), file=sys.stdout, unit = "batch") as prbar: # pregress bar
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    return metrics


In [ ]:
def fit(
    model,
    epochs,
    train_data_loader,
    validation_data_loader,
    optimizer,
    criterion,
    device="cuda"
):
    all_train_losses = []
    epoch_train_losses = []
    epoch_eval_losses = []
    for epoch in range(epochs):
        # Train step
        print(f"Train Epoch: {epoch}")
        train_metrics, one_epoch_train_losses = train_epoch(
            model=model,
            data_loader=train_data_loader,
            optimizer=optimizer,
            return_losses=True,
            criterion=criterion,
            device=device
        )
        # Save Train losses
        all_train_losses.extend(one_epoch_train_losses) # добавляет в конец списка как отдельные элементы
        epoch_train_losses.append(train_metrics["loss"]) # добавляет в конец список как список
        # Eval step
        print(f"Validation Epoch: {epoch}")
        with torch.no_grad():
            validation_metrics = validate(
                model=model,
                data_loader=validation_data_loader,
                criterion=criterion
            )
        # Save eval losses
        epoch_eval_losses.append(validation_metrics["loss"]) # добавляет в конец список как список

In [ ]:
##nn.NLLLoss2d

## Fit Pnasnet5large

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_Pnasnet5large = torch.optim.Adam(model_pnasnet5large.last_linear.parameters(), lr=0.001)
EPOCH_Pnasnet5large = 120
print(device)

In [ ]:
fit(model_pnasnet5large, EPOCH_Pnasnet5large , train_dataloader, validation_dataloader, optimizer_Pnasnet5large, criterion, device=device)

## Fit DenseNet 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_densenet  = torch.optim.Adam(model_densenet.classifier.parameters(), lr=0.001)
EPOCH_densenet  = 120

In [ ]:
fit(model_densenet, EPOCH_densenet , train_dataloader, validation_dataloader, optimizer_densenet, criterion, device=device)

## Fit EffNet_b7

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model._fc.parameters(), lr=0.001)
EPOCH = 100
print(device)

In [ ]:
fit(model, EPOCH , train_dataloader, validation_dataloader, optimizer, criterion, device=device)

## Fit VGG19

In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model._fc.parameters(), lr=0.07, momentum=0.9)
optimizer_vgg19 = torch.optim.Adam(model_vgg19.classifier.parameters(), lr=0.001)
EPOCH_vgg19 = 150

In [ ]:
fit(model_vgg19,EPOCH_vgg19, train_dataloader, validation_dataloader, optimizer_vgg19, criterion, device=device)

## Fit Resnet50

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_resnet50 = torch.optim.Adam(model_resnet50.fc.parameters(), lr=0.001)
EPOCH_resnet50 = 120

In [ ]:
fit(model_resnet50,EPOCH_resnet50, train_dataloader, validation_dataloader, optimizer_resnet50, criterion, device=device)

## Fit Inception

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_inception = torch.optim.Adam(model_inception.fc.parameters(), lr=0.001)
EPOCH_inception = 120

In [ ]:
fit(model_inception,EPOCH_inception, train_dataloader, validation_dataloader, optimizer_inception, criterion, device=device)

## Fit Google

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_google = torch.optim.Adam(model_googlenet.fc.parameters(), lr=0.001)
EPOCH_google = 80

In [ ]:
fit(model_googlenet,EPOCH_google, train_dataloader, validation_dataloader, optimizer_google, criterion, device=device)

In [ ]:
# # Save the Modle to file in the current working directory

# Pkl_Filename = "WoodHack.pkl"  

# with open(Pkl_Filename, 'wb') as file:  
#     pickle.dump(model, file)

In [ ]:
# Pkl_Filename2 = "WoodHack_full.pkl"  
# with open(Pkl_Filename2, 'wb') as file:  
#     pickle.dump(model_full, file)


## Сохранение модели

## Pnasnet

In [ ]:
PATH_pnas = "pnas.pth"
os.chdir("/kaggle/working/")
torch.save(model_pnasnet5large.state_dict(), PATH_pnas ) #state_dict - получить текущее состояние Optimizer. Для адаптивных методов тут будут храниться аккумулированные квадраты градиентов

## Densenet

In [ ]:
PATH_levit = "dencenet.pth"
os.chdir("/kaggle/working/")
torch.save(model_densenet.state_dict(), PATH_levit ) #state_dict - получить текущее состояние Optimizer. Для адаптивных методов тут будут храниться аккумулированные квадраты градиентов

## Efficcient_net_B7

In [ ]:
PATH = "effnetb7.pth"
os.chdir("/kaggle/working/")
torch.save(model.state_dict(), PATH ) #state_dict - получить текущее состояние Optimizer. Для адаптивных методов тут будут храниться аккумулированные квадраты градиентов

## Resnet50

In [ ]:
PATH2 = "resnet50.pth"
os.chdir("/kaggle/working/")
torch.save(model_resnet50.state_dict(), PATH2 ) #state_dict - получить текущее состояние Optimizer. Для адаптивных методов тут будут храниться аккумулированные квадраты градиентов

## VGG19

In [ ]:
PATH3 = "vgg19.pth"
os.chdir("/kaggle/working/")
torch.save(model_vgg19.state_dict(), PATH3 ) #state_dict - получить текущее состояние Optimizer. Для адаптивных методов тут будут храниться аккумулированные квадраты градиентов

## Google

In [ ]:
PATH3 = "google.pth"
os.chdir("/kaggle/working/")
torch.save(model_googlenet.state_dict(), PATH3 ) #state_dict - получить текущее состояние Optimizer. Для адаптивных методов тут будут храниться аккумулированные квадраты градиентов

## Inception

In [ ]:
PATH3 = "inception.pth"
os.chdir("/kaggle/working/")
torch.save(model_inception.state_dict(), PATH3 ) #state_dict - получить текущее состояние Optimizer. Для адаптивных методов тут будут храниться аккумулированные квадраты градиентов

In [ ]:
# Load the Model back from file
# Pkl_Filename = "WoodHack.pkl" 
# with open(Pkl_Filename, 'rb') as file:  
#     model = pickle.load(file)

# model

In [ ]:
# # Load the Model_full back from file
# Pkl_Filename2 = "WoodHack_full.pkl"  
# with open(Pkl_Filename2, 'rb') as file:  
#     model_full = pickle.load(file)

# model_full

In [ ]:
# import requests
# from PIL import Image
# image = '../input/woodhack/test/1.png'
# url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT76mSMtKQWGstcqGi-0kPWJyVBqz8RCp8SuplMipkidRY0z9Mc&usqp=CAU'

# # response = requests.get(image, stream = True)
# img = Image.open("../input/woodhack/test1/test/")
# plt.imshow(img)


In [ ]:
# import requests
# from PIL import Image

# def im_convert(tensor):  
#   image = tensor.cpu().clone().detach().numpy() # This process will happen in normal cpu.
#   image = image.transpose(1, 2, 0)
#   image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
#   image = image.clip(0, 1)
#   return image
# for i in range(1,250):
#     path = "../input/woodhack/test1/test/" + str(i) + ".png"
#     img = Image.open(path)
#     #plt.imshow(img)
#     img = transformations(img)
#     img = img[None, :]
#     #plt.imshow(im_convert(img))
#     #print(img.size())
#     #predicted = model(img)
#     print(img.size())


# def im_convert(tensor):  
#   image = tensor.cpu().clone().detach().numpy() # This process will happen in normal cpu.
#   image = image.transpose(1, 2, 0)
#   image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
#   image = image.clip(0, 1)
#   return image

# img = transformations(img)  # applying the transformations on new image as our model has been trained on these transformations
# plt.imshow(im_convert(img)) # convert to numpy array for plt

In [ ]:
#  image = img.to(device).unsqueeze(0) # put inputs in device as our model is running there
#  output = model(image)
#  _, pred = torch.max(output, 1)
#  print(classes[pred.item()])


## Pnas Prediction


In [ ]:
test_pnas = []
from PIL import Image
for i in range(1,250):
    path = "../input/neurowood/testset/" + str(i) + '.jpg'
   #else:
        #path = "../input/woodhack/test1/test/" + str(i) + '.JPG'
    img = Image.open(path)
    #plt.imshow(img)
    img = transformations(img)
    img = img[None, :]
    img = img.to(device)
    predicted = model_pnasnet5large(img)
    #print(predicted,img.size())
    _, pred = torch.max(predicted, 1)
    test_pnas.append(classes[pred.item()])
print(test_pnas)

## Levit Prediction

In [ ]:
test_dencenet = []
from PIL import Image
for i in range(1,250):
    path = "../input/neurowood/testset/" + str(i) + '.jpg'
   #else:
        #path = "../input/woodhack/test1/test/" + str(i) + '.JPG'
    img = Image.open(path)
    #plt.imshow(img)
    img = transformations(img)
    img = img[None, :]
    img = img.to(device)
    predicted = model_densenet(img)
    #print(predicted,img.size())
    _, pred = torch.max(predicted, 1)
    test_dencenet.append(classes[pred.item()])
print(test_dencenet)

## Efficcient_net_B7 predict

In [ ]:
test = []
from PIL import Image
for i in range(1,250):
    path = "../input/neurowood/testset/" + str(i) + '.jpg'
   #else:
        #path = "../input/woodhack/test1/test/" + str(i) + '.JPG'
    img = Image.open(path)
    #plt.imshow(img)
    img = transformations(img)
    img = img[None, :]
    img = img.to(device)
    predicted = model(img)
    #print(predicted,img.size())
    _, pred = torch.max(predicted, 1)
    test.append(classes[pred.item()])
print(test)

## Resnet50 Predictions

In [ ]:
test_resnet50 = []
from PIL import Image
for i in range(1,250):
    path = "../input/neurowood/testset/" + str(i) + '.jpg'
   #else:
        #path = "../input/woodhack/test1/test/" + str(i) + '.JPG'
    img = Image.open(path)
    #plt.imshow(img)
    img = transformations(img)
    img = img[None, :]
    img = img.to(device)
    predicted = model_resnet50(img)
    #print(predicted,img.size())
    _, pred = torch.max(predicted, 1)
    test_resnet50.append(classes[pred.item()])
print(test_resnet50)

## VGG19 Predictions

In [ ]:
test_vgg19 = []
from PIL import Image
for i in range(1,250):
    path = "../input/neurowood/testset/" + str(i) + '.jpg'
   #else:
        #path = "../input/woodhack/test1/test/" + str(i) + '.JPG'
    img = Image.open(path)
    #plt.imshow(img)
    img = transformations(img)
    img = img[None, :]
    img = img.to(device)
    predicted = model_vgg19(img)
    #print(predicted,img.size())
    _, pred = torch.max(predicted, 1)
    test_vgg19.append(classes[pred.item()])
print(test_vgg19)

## Inception Predictions

In [ ]:
test_inception = []
from PIL import Image
for i in range(1,250):
    path = "../input/neurowood/testset/" + str(i) + '.jpg'
   #else:
        #path = "../input/woodhack/test1/test/" + str(i) + '.JPG'
    img = Image.open(path)
    #plt.imshow(img)
    img = transformations(img)
    img = img[None, :]
    img = img.to(device)
    predicted = model_inception(img)
    #print(predicted,img.size())
    _, pred = torch.max(predicted, 1)
    test_inception.append(classes[pred.item()])
print(test_inception)

## Google Predictions

In [ ]:
test_google = []
from PIL import Image
for i in range(1,250):
    path = "../input/neurowood/testset/" + str(i) + '.jpg'
   #else:
        #path = "../input/woodhack/test1/test/" + str(i) + '.JPG'
    img = Image.open(path)
    #plt.imshow(img)
    img = transformations(img)
    img = img[None, :]
    img = img.to(device)
    predicted = model_googlenet(img)
    #print(predicted,img.size())
    _, pred = torch.max(predicted, 1)
    test_google.append(classes[pred.item()])
print(test_google)

In [ ]:
# test = []
# for images, lables in test_dataloader: # батч размером 1! # lable =0 тк у меня один класс
#     images = images.to(device)
#     lables = lables.to(device)
#     predicted = model(images)
#     print(predicted,images.size(), lables)
#     print()
#     _, pred = torch.max(predicted, 1)
#     test.append(classes[pred.item()])
# print(test)

In [ ]:
# test_full = []
# for images, lables in test_dataloader:
#     images = images.to(device)
#     lables = lables.to(device)
#     predicted = model(images)
#     _, pred = torch.max(predicted, 1)
#     test_full.append(classes[pred.item()])
# print(test_full)

In [ ]:
# np.size(test)

In [ ]:
df = pd.DataFrame({"id" : range(1,250),
                   "eff": test,
                   "res152": test_resnet50,
                   "vgg19": test_vgg19,
                   "densenet": test_dencenet,
                   "pnas": test_pnas,
                   "google":test_google,
                   "inception":test_inception
                  })

In [ ]:
df['rez'] = df.mode(axis='columns')[0]

In [ ]:
df.head(60)

In [ ]:
df.tail(60)

In [ ]:
# df['sum'] = df['eff'] + df['res50'] + df['vgg19']
# df['sum'] = df['sum'].map({1:0, 0:0, 2:1, 3:1})
ansamble_pred = df['rez'].to_numpy()

In [ ]:
df = pd.DataFrame({"id" : range(1,250),
                   "class": ansamble_pred
                  })

In [ ]:
# df = pd.DataFrame({"id" : range(1,250),
#                    "class": test
#                   })

In [ ]:
# df["class"]

In [ ]:
df.tail(50)

In [ ]:
# df_full.tail(50)

In [ ]:
print('Generating submission file...')
#os.chdir("./")
df.to_csv('submission_Ansamble7.csv',index=False)

In [ ]:
# df_full.to_csv('submission_effnet7new2.csv',index=False)